# Final Code

Importing libraries

In [27]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import datetime as dt

from sklearn import tree
from sklearn.metrics import make_scorer, accuracy_score
from sklearn.metrics import f1_score

Importing datasets

In [28]:
data_train = pd.read_csv('./data_set/Hotel-A-train.csv')
data_test = pd.read_csv('./data_set/Hotel-A-validation.csv')
data_out_or = pd.read_csv('./data_set/Hotel-A-test.csv')

Defining data preprocessing functions

In [29]:
def prepare(df,train):
    df=df.dropna()
    df=df.drop_duplicates()
    df = prepDates(df)
    df = binAge(df)
    df = makeCats(df)
    df = binRoomRate(df)
    df=df.dropna()
    df=df.drop_duplicates()
    if(train):
        y = df[["Reservation_Status"]]
        y = catStatus(y)
        return(df,y)
    else:
        ids = ids = df[["Reservation-id"]]
        return (df,ids)

def prepDates(df):
    df[["Expected_checkin","Expected_checkout","Booking_date"]]=df[["Expected_checkin","Expected_checkout","Booking_date"]].astype('datetime64[ns]')
    df[["Expected_checkin","Expected_checkout","Booking_date"]].dtypes
    df[["Staying"]] = df["Expected_checkout"]-df["Expected_checkin"]
    df[["Before"]] = df["Expected_checkin"]-df["Booking_date"]
    df[["Before"]] = (df[["Before"]] / np.timedelta64(1, 'D')/7).astype('int')
    df[["Month"]] = pd.DatetimeIndex(df["Expected_checkin"]).month
    return df;

def binAge(df):
    df.Age = df.Age.fillna(-0.5)
    bins = (-1, 0, 5, 12, 18, 25, 35, 60, 120)
    group_names = ['Unknown', 'Baby', 'Child', 'Teenager', 'Student', 'Young Adult', 'Adult', 'Senior']
    categories = pd.cut(df.Age, bins, labels=group_names)
    df[['Age_cat']] = categories.cat.codes
    return df

def binRoomRate(df):
    df.Room_Rate = df.Room_Rate.fillna(1)
    bins = (100,150,200,250,300)
    group_names = [1,2,3,4];
    categories = pd.cut(df.Room_Rate, bins, labels=group_names)
    df.Room_Rate = categories.cat.codes
    return df

def binBefore(df):
    df.Before = df.Before.fillna(1)
    bins = (-1000,0,10,20,50,100,200,500,1000)
    group_names = [0,1,2,3,4,5,6,7];
    categories = pd.cut(df.Before, bins, labels=group_names)
    df.Before = categories
    df['Before'] = df['Before'].cat.codes
    return df

def makeCats(x):
    x[["Gender","Ethnicity","Educational_Level","Income","Country_region","Hotel_Type","Meal_Type","Visted_Previously","Previous_Cancellations","Deposit_type","Booking_channel","Required_Car_Parking","Use_Promotion","Staying","Discount_Rate"]]=x[["Gender","Ethnicity","Educational_Level","Income","Country_region","Hotel_Type","Meal_Type","Visted_Previously","Previous_Cancellations","Deposit_type","Booking_channel","Required_Car_Parking","Use_Promotion","Staying","Discount_Rate"]].astype("category")
    cat_columns =["Gender_cat","Ethnicity_cat","Educational_Level_cat","Income_cat","Country_region_cat","Hotel_Type_cat","Meal_Type_cat","Visted_Previously_cat","Previous_Cancellations_cat","Deposit_type_cat","Booking_channel_cat","Required_Car_Parking_cat","Use_Promotion_cat","Staying_cat","Discount_Rate_cat"]
    columns=["Gender","Ethnicity","Educational_Level","Income","Country_region","Hotel_Type","Meal_Type","Visted_Previously","Previous_Cancellations","Deposit_type","Booking_channel","Required_Car_Parking","Use_Promotion","Staying","Discount_Rate"]
    for i in range(len(cat_columns)):
        x[cat_columns[i]]=x[columns[i]].cat.codes
    return x

def catStatus(df):
    cleanups ={"Reservation_Status":{"Check-In":1,"Canceled":2,"No-Show":3}}
    df = df.replace(cleanups)
    return df

def drop(df,columns):
    df = df.drop(columns = columns)
    return df
   
    
def out(df,columns,train):
    x,y = prepare(df,train)
    #x = binBefore(x)
    x = drop(x,columns)
    return (x,y)


Preprocessing data

In [30]:
columns_train=["Reservation-id","Reservation_Status","Expected_checkin","Expected_checkout","Booking_date","Age","Gender","Ethnicity","Educational_Level","Income","Country_region","Hotel_Type","Meal_Type","Visted_Previously","Previous_Cancellations","Deposit_type","Booking_channel","Required_Car_Parking","Use_Promotion","Staying","Discount_Rate"]#,"Room_Rate"]
columns_out=["Reservation-id","Expected_checkin","Expected_checkout","Booking_date","Age","Gender","Ethnicity","Educational_Level","Income","Country_region","Hotel_Type","Meal_Type","Visted_Previously","Previous_Cancellations","Deposit_type","Booking_channel","Required_Car_Parking","Use_Promotion","Staying","Discount_Rate"]#,"Room_Rate"]

data_train_x,data_train_y = out(data_train,columns_train,True)
data_test_x,data_test_y =out(data_test,columns_train,True)
data_out,ids = out(data_out_or,columns_out,False)
data_train_x

,Adults,Children,Babies,Room_Rate,Before,Month,Age_cat,Gender_cat,Ethnicity_cat,Educational_Level_cat,...,Hotel_Type_cat,Meal_Type_cat,Visted_Previously_cat,Previous_Cancellations_cat,Deposit_type_cat,Booking_channel_cat,Required_Car_Parking_cat,Use_Promotion_cat,Staying_cat,Discount_Rate_cat
0,2,2,0,2,5,7,6,0,2,1,...,1,0,0,0,0,2,1,1,0,2
1,3,3,0,1,5,7,6,0,2,3,...,1,0,0,0,2,2,1,0,0,0
2,3,3,0,0,0,7,6,0,3,1,...,1,0,0,0,0,2,1,0,3,0
3,4,3,0,0,1,7,4,1,0,0,...,0,0,0,0,2,0,1,1,0,1
4,1,1,0,2,1,7,7,0,2,2,...,2,0,0,0,0,1,0,1,0,2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
27494,1,2,1,-1,5,8,6,0,0,3,...,1,2,1,1,0,2,1,1,0,2
27495,2,1,0,1,18,8,6,0,1,0,...,0,0,0,0,2,0,1,1,0,7
27496,2,1,1,2,1,9,6,0,2,2,...,0,2,0,0,0,1,1,1,0,1
27497,3,2,0,1,14,8,7,1,0,0,...,0,1,0,0,0,2,0,1,0,4


In [ ]:
Defining the classifier and training the model.

In [31]:
clf = tree.DecisionTreeClassifier()
clf = clf.fit(data_train_x, data_train_y.values.ravel())

Get scores for validation dataset

In [32]:
pred = clf.predict(data_test_x)
print(accuracy_score( data_test_y,pred))
print(f1_score(data_test_y, pred, average='macro'))

0.4783557657329938
0.34040222762051137


Predicting and saving the output

In [33]:
pred = clf.predict(data_out)
ids[['Reservation_Status']]=pred
ids.to_csv('AGNI_CODE_HUNTERs.csv',index=False)
print(ids.sample(20))

      Reservation-id  Reservation_Status
4007        63204078                   1
4099        24604581                   2
3113        42655220                   1
1362        21915580                   1
3712        14975639                   1
4253        10772755                   1
836         73940542                   1
4111        74758107                   3
4106        95097963                   1
1905        49896464                   2
2913        29303076                   3
169         31891030                   2
3523        84097363                   1
627         18739601                   1
3215        33488678                   2
3383        50285280                   1
2229        35209776                   2
2970        35223263                   1
2162        16520129                   1
4035        46285920                   2
